In [ ]:
#!pip install box2d-py
#!pip install "stable-baselines3[extra]>=2.0.0a4"
#!conda install -c conda-forge gcc=12.1.0

In [1]:
import gymnasium as gym
NUM_ENVS = 5
#envs = gym.vector.make('ALE/Breakout-v5', num_envs=NUM_ENVS)#, render_mode='human')
envs = gym.vector.make('LunarLander-v2', num_envs=NUM_ENVS)#, render_mode='human')


In [2]:
import tensorflow as tf
import numpy as np
import random

# Define the CNN model
def create_model():
    inputs = tf.keras.Input(shape=(8,))
    x = tf.keras.layers.Dense(8, activation="relu")(inputs)
    x = tf.keras.layers.Dense(8, activation="relu")(x)
    x = tf.keras.layers.Dense(8, activation="relu")(x)
    outputs = tf.keras.layers.Dense(4, activation="relu")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="LunarLander")
    return model

# Create the model
model = create_model()
optimizer = tf.keras.optimizers.Adam()
# Copy the model
target_network = tf.keras.models.clone_model(model)



2023-06-12 16:06:12.697697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 16:06:14.226247: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
SAMPLE_SIZE = 10
EPSILON = 0.2

class ERP():
    def __init__(self):
        obs, info = envs.reset()
        self.observation = obs
        print("init obs:")
        print(self.observation)

    def generate_samples(self):
        # sample s, a, r, s' and add it to replay buffer
        for i in range(SAMPLE_SIZE):
            print("old obs:")
            print(self.observation)
            action = np.argmax(model(self.observation), axis=0)
            action = [random.choice(range(4)) if random.random() < EPSILON else a for a in action] #epsilon greedy policy
            new_observation, reward, terminated, truncated, info = envs.step(action)
            old_observation = self.observation
            #if terminated.any() or truncated.any(): <- no need in vectorized env
            #    observation, info = envs.reset()
            # extract individual environments
            self.observation = new_observation
            print("new obs:")
            print(self.observation)
            for j in range(NUM_ENVS):
                yield (old_observation[j], action[j] , reward[j], new_observation[j])
            

Erp = ERP()
#next(Erp.generate_samples())

sig = (tf.TensorSpec(shape=(8), dtype = tf.uint8), tf.TensorSpec(shape=(), dtype = tf.int32), tf.TensorSpec(shape=(), dtype = tf.float32), tf.TensorSpec(shape=(8), dtype = tf.uint8))
ds = tf.data.Dataset.from_generator(Erp.generate_samples, output_signature = sig)
ds.prefetch(buffer_size=NUM_ENVS*SAMPLE_SIZE).cache().shuffle(buffer_size = NUM_ENVS*SAMPLE_SIZE, reshuffle_each_iteration = True)

        

init obs:
[[ 0.00496216  1.4089434   0.5025958  -0.08785639 -0.0057431  -0.11384554
   0.          0.        ]
 [ 0.00420198  1.4172679   0.42560512  0.2821156  -0.00486231 -0.09640589
   0.          0.        ]
 [-0.00745811  1.4037193  -0.75544393 -0.32006347  0.0086489   0.17111933
   0.          0.        ]
 [ 0.00571785  1.4162394   0.57914466  0.23640251 -0.00661882 -0.13118501
   0.          0.        ]
 [-0.00427132  1.4070904  -0.4326521  -0.1702099   0.00495615  0.09800202
   0.          0.        ]]


NameError: name 'generate_samples' is not defined

In [ ]:
next(Erp.generate_samples())

old obs:
[[ 0.00281448  1.3939519   0.10158356 -0.23564729 -0.00256966 -0.01017522
   0.          0.        ]]
new obs:
[[ 0.00382519  1.3880495   0.10158485 -0.26233083 -0.00307866 -0.01018112
   0.          0.        ]]


(array([ 0.00281448,  1.3939519 ,  0.10158356, -0.23564729, -0.00256966,
        -0.01017522,  0.        ,  0.        ], dtype=float32),
 0,
 -1.9311562311576438,
 array([ 0.00382519,  1.3880495 ,  0.10158485, -0.26233083, -0.00307866,
        -0.01018112,  0.        ,  0.        ], dtype=float32))

In [6]:
for elem in ds:
    print(elem)

2023-06-12 15:30:27.092536: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-12 15:30:27.121381: W tensorflow/core/framework/op_kernel.cc:1818] UNKNOWN: UnboundLocalError: cannot access local variable 'observation' where it is not associated with a value
Traceback (most recent call last):

  File "/home/mindset/miniconda3/envs/drl/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/home/mindset/miniconda3/envs/drl/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/home/mindset/miniconda3/envs/drl/lib/python3.11/site-packages/tens

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_4_device_/job:localhost/replica:0/task:0/device:CPU:0}} UnboundLocalError: cannot access local variable 'observation' where it is not associated with a value
Traceback (most recent call last):

  File "/home/mindset/miniconda3/envs/drl/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/home/mindset/miniconda3/envs/drl/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/home/mindset/miniconda3/envs/drl/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/tmp/ipykernel_12165/887646877.py", line 7, in generate_samples
    action =

In [5]:
tf.data.Dataset.sample_from_datasets(datasets, weights=[float(len(data))/float(len(datasets)) for data in datasets], stop_on_empty_dataset=False)

<_FlatMapDataset element_spec=(TensorSpec(shape=(8,), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(8,), dtype=tf.uint8, name=None))>

In [ ]:
from collections import deque
import numpy as np
import random

max_steps = 100000
N = 10
K = 10
EPSILON = 0.2
MINI_BATCH_SIZE = 10
GAMMA = 0.5
TAU = 0.9
converged = False

# Create the replay buffer
buffer_size = 10000  # Define the size of the replay buffer
buffer = deque(maxlen=buffer_size)

observation, info = envs.reset()

print(observation.shape)
print(model(observation))


loss_value = 0


while not converged and max_steps > 0:
    target_network.set_weights((1 - TAU) * np.array(target_network.get_weights()) + TAU * np.array(model.get_weights())) #polyak averaging
    for n in range(N):
        
        print(f"n: {n}", loss_value)


        # sample s, a, r, s' and add it to replay buffer
        action = np.argmax(model(observation), axis=0)
        action = [random.choice(range(4)) if random.random() < EPSILON else a for a in action]
        new_observation, reward, terminated, truncated, info = envs.step(action)
        if terminated.any() or truncated.any():
            observation, info = envs.reset()
        # extract individual environments
        for j in range(NUM_ENVS):
            buffer.append((observation[j], action[j] , reward[j], new_observation[j]))
        observation = new_observation



        for k in range(K):
            # sample minibatch from batch
            mini_batch = random.sample(buffer, MINI_BATCH_SIZE if MINI_BATCH_SIZE<=len(buffer) else len(buffer))
            mb_observations = []
            mb_actions = []
            mb_rewards = []
            mb_new_observations = []
            for o, a, r, no in mini_batch:
                mb_observations.append(o)
                mb_actions.append(a)
                mb_rewards.append(r)
                mb_new_observations.append(no)
            # Convert lists to numpy arrays or TensorFlow tensors
            #print("actions")
            #print(mb_actions)
            mb_observations = np.array(mb_observations)
            mb_actions = np.array(mb_actions)
            mb_rewards = np.array(mb_rewards)
            mb_new_observations = np.array(mb_new_observations)

            #print("test")
            #print(mb_observations.shape)
            #print(model(mb_observations))
            # obtain target Q-values (labels)
            target_q_values = GAMMA * np.max(target_network(mb_new_observations), axis=1) + mb_rewards
            #print("Targets")
            #print(target_q_values)
            # Perform forward and backward pass
            with tf.GradientTape() as tape:
                predictions = model(mb_observations)
                #print("Predictions")
                #print(predictions)
                selected_q_values = tf.gather(predictions, mb_actions, batch_dims=1)
                #print("Selected")
                #print(selected_q_values)
                loss_value = tf.reduce_mean(tf.square(target_q_values - selected_q_values))
            gradients = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            
    max_steps -= 1




#envs.close()

In [5]:
tf.gather([[1,2,3],[4,5,6],[7,8,9]], [1,0,2], batch_dims=1)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 4, 9], dtype=int32)>